# POS-EGNN 

## Setup

In [ ]:
from ase.io import read
from ase.md.langevin import Langevin
from ase import units

from posegnn.calculator import PosEGNNCalculator

import torch

In [ ]:
device = "cpu"
torch.set_float32_matmul_precision("high")

## Feature Extraction

In [ ]:
calculator = PosEGNNCalculator("pos-egnn.v1-6M.ckpt", device=device, compute_stress=False)
atoms = read("inputs/3BPA.xyz", index=0)
data = calculator._build_data(atoms)
embeddings = calculator.model.forward(data.to(device))["embedding_0"][:,:,:, 1].squeeze(2)

embeddings.shape

## Inference

In [ ]:
out = calculator.model.compute_properties(data, compute_stress=False)
energy = out["total_energy"]
force = out["force"]

In [ ]:
energy, force

In [ ]:
atoms.calc = calculator
atoms.get_total_energy(), atoms.get_forces()

## Molecular Dynamics Simulation

In [ ]:
dyn = Langevin(atoms=atoms, friction=0.005, temperature_K=310, timestep=0.5 * units.fs)

def write_frame():
    dyn.atoms.write("output.xyz", append=True)

dyn.attach(write_frame, interval=1)
dyn.run(100)